In [1]:
import sys
from typing import Literal

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-12 22:48:48.178471: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 22:48:48.184980: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-12 22:48:48.220340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 22:48:48.220388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 22:48:48.221556: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [29]:
n_samples = 32
sampler = EquationSampler(BASE_TOKENS, 16, 2)
(
    sequences,
    entropies,
    log_probs,
    counters,
    lengths,
    sequence_mask,
) = sampler.sample_sequence(n_samples)

In [30]:
sequence = sequences[0]
length = lengths[0]

In [31]:
def build_tree(sequence):
    value = sequence[0]
    token = BASE_TOKENS[value]
    root = Node.from_token(token, parent=None)

    current_arity = root.arity
    previous = root

    for value in sequence[1:]:
        current_arity -= 1
        token = BASE_TOKENS[value]
        child = Node.from_token(token)
        current_arity += child.arity
        
        previous.add_child(child)
        
        previous = child
        while not previous.remaining_children():
            previous = previous.parent
            if previous is None:
                break
    return root

TODO constraint avoid x / x

In [34]:
build_tree(sequence[0:length])._recursive_expr_repr()

'log(var_x / const) + exp(var_x / cos(const / const))'

In [ ]:
for value in sequence:
    print(BASE_TOKENS[value])

In [14]:
tokens = [token.symbol for token in BASE_TOKENS]
items = ['cos', '+', 'const', 'exp', 'var_x']
items = [tokens.index(item) for item in items]

In [15]:
items

[5, 0, 8, 6, 9]

In [25]:
build_tree(items)

cos(const + exp(var_x))

In [ ]:
root = Node.from_token(BASE_TOKENS[5])
previous = root

In [ ]:
previous

In [ ]:
child = Node.from_token(BASE_TOKENS[0])
previous.add_child(child)
previous = child

In [ ]:
previous

In [ ]:
child = Node.from_token(BASE_TOKENS[8])
previous.add_child(child)
previous = child

In [ ]:
previous

In [ ]:
previous.parent

In [ ]:
previous = previous.parent

In [ ]:
child.remaining_children()

In [ ]:
build_tree(sequence[:length-1])